# LOB 

Here I design, simulate and analyze a limit order book similar to those used in real financial markets, with a focus on microstructure dynamics, order-flow modelling, and strategy backtesting.

### Overview 
1) LOB Structure
    - Orders: market, limit, cancel
    - Book: price-time priority, bid-ask spread, mid-price
    - Order matching and execution 
2) Order-flow simulation 
    - Synthetic order arrivals (Poisson model)
    - Define distributions for order size, arrival intensity. 
3) Analysis of statistical properties 
    - Spread, volatility, mid-price variance.
    - Market impact and slippage
    - Flow Autocorrelation 
4) Basic Strategy implementation 
    - Market maker arbitrage strategy 
    - Backtest strategy: average returns, sharpe ratio, capital allocation.
  
### 1 — LOB structure 

Our order book will be L3 Market by Order. This means we will keep track of every order: it's price, status, priority, and side. The most important implementation decision is how we will store the orders such that the most common tasks — viewing and removing/matching the best bids and asks is — efficient.

**Price-Time priority** will be used to execute trades. The data structure that best encodes this is a heap of queues: queues manage time-priority while the heap manages price priority. Cancellations pose a problem because they require ID lookup which can be slow, O(N), in heaps. In order to improve this we use a dictionary to map prices to queues which allow for O(1) price lookup and then linear search along a queue assumed to be small. 

In [ ]:
from collections import deque
import heapq as hq

class Order: 
    _id_counter = 0 

    def __init__(self, price, volume, is_bid):
        self.price = price 
        self.volume = volume 
        self.is_bid = is_bid
        # self.status = "active" # not needed for non-lazy order-cancellation.
        self.id = Order._id_counter
        # Increment class variable to ensure Unique ID.
        Order._id_counter += 1

class PriceLevel:
    def __init__(self, price):
        self.price = price 
        self.orders = deque()
    
    def add(self, order):
        self.orders.append(order)

    def top(self):
        if not self.orders:
            return None
        return self.orders[0] # O(1)

    def pop(self):
        if not self.orders:
            raise IndexError("Price level is empty.")
        return self.orders.popleft() # O(1)

    def cancel(self, order):
        try:
            self.orders.remove(order) # O(n)
        except ValueError:
            raise ValueError("Order not found at this price level.")
        
    def is_empty(self):
        return True if len(self.orders) == 0 else False

class PriceBook:
    def __init__(self, is_bid_side):
        self._price_levels = {}
        self._heap = []
        self.is_bid_side = is_bid_side

    def add(self, order): 
        price = order.price

        if price not in self._price_levels:
            self._price_levels[price] = PriceLevel(price)
            # Use negative price for max-heap behavior on bid side.
            heap_price = -price if self.is_bid_side else price
            hq.heappush(self._heap, heap_price)

        self._price_levels[price].add(order)
    
    def cancel(self, order):
        price = order.price
        if price not in self._price_levels:
            raise ValueError(f"Price {price} not found in PriceBook.")

        try:
            self._price_levels[price].cancel(order)
            # Trying to remove the price level from the heap here would be O(N)
        except ValueError as e:
            # Propagate the original error — don't swallow it
            raise ValueError(f"Failed to cancel order {order.id}: {e}") from e

    def best_price(self):
        if not self._price_levels: 
            return None
        else:
            # Since I didn't remove empty price levels upon order cancellation I have to handle it here
            while self._heap:
                best_price = - self._heap[0] if self.is_bid_side else self._heap[0]
                # Remove empty price levels
                if self._price_levels[best_price].is_empty():
                    hq.heappop(self._heap)
                    del self._price_levels[best_price]
                else: 
                    break

            if self._heap:
                return best_price
            else:
                return None
            
    def peek_best_order(self):
        best_price = self.best_price()
        if best_price:
            return self._price_levels[best_price].top()
        else:
            return None
        
    def fill_volume(self, volume, price): 
        #to do 
         
    # delete when fill_volume is complete 
    def pop_best_order(self):
        best_price = self.best_price()
        if best_price: 
            return self._price_levels[best_price].pop()
        else:
            return None
        
class OrderBook:
    def __init__(self):
        self.bids = PriceBook(is_bid_side=True)
        self.asks = PriceBook(is_bid_side=False)
        self.order_map = dict()
    
    @property
    def best_bid(self):
        return self.bids.best_price()
    
    @property
    def best_ask(self):
        return self.asks.best_price()
    
    @property 
    def spread(self):
        if not (self.best_bid and self.best_ask):
            return None
        else: 
            return self.best_ask-self.best_bid
    
    @property 
    def mid_price(self):
        if not (self.best_bid and self.best_ask):
            return None
        else: 
            return (self.best_ask + self.best_bid) / 2
    
    def process_order(self, order):
        self.order_map[order.id] = order
        # Order Matching Logic:
    
    def cancel(self, order_id):
        try: 
            order = self.order_map[order_id]
        except KeyError as e: 
            raise KeyError(f"Order with id {order_id} could not be found: {e}")

        if order.is_bid:
            self.bids.cancel(order)
        else:
            self.asks.cancel(order)

        del self.order_map[order_id]

We generate some tests with an LLM to make sure that the Order Book works as we want it to.

In [31]:
import unittest
class TestLOB(unittest.TestCase):
    
    def setUp(self):
        # Reset the order ID counter before each test for predictable IDs
        Order._id_counter = 0 
        self.lob = OrderBook()
        
    def test_price_time_priority_and_best_price(self):
        """Tests adding orders and verifying the best price and time priority."""
        
        # --- Add Bids (Max Price Priority, FIFO Time Priority) ---
        # Bid 1: Best Price, First in Time (ID 0)
        order_b1 = Order(price=10.0, volume=100, is_bid=True) 
        self.lob.add(order_b1)
        # Bid 2: Second Best Price (ID 1)
        order_b2 = Order(price=9.95, volume=50, is_bid=True) 
        self.lob.add(order_b2)
        # Bid 3: Same Price as B1, Second in Time (ID 2)
        order_b3 = Order(price=10.0, volume=200, is_bid=True) 
        self.lob.add(order_b3)

        # 1. Verify Best Bid Price
        self.assertEqual(self.lob.bids.best_price(), 10.0, "Best bid price should be 10.0")

        # 2. Verify Time Priority (FIFO pop)
        popped_order = self.lob.bids.pop_best_order()
        self.assertEqual(popped_order.id, order_b1.id, "First order popped should be the first one added at 10.0 (ID 0)")
        
        # 3. Verify Next in Queue
        next_order = self.lob.bids.pop_best_order()
        self.assertEqual(next_order.id, order_b3.id, "Second order popped should be the second one added at 10.0 (ID 2)")

        # 4. Verify Heap Clean Up (10.0 level is now empty, best price moves to 9.95)
        self.assertEqual(self.lob.bids.best_price(), 9.95, "Best bid should now move to 9.95")
        
        # --- Add Asks (Min Price Priority) ---
        # Ask 1: Best Price (ID 3)
        order_a1 = Order(price=10.10, volume=100, is_bid=False) 
        self.lob.add(order_a1)
        # Ask 2: Second Best Price (Higher) (ID 4)
        order_a2 = Order(price=10.15, volume=50, is_bid=False) 
        self.lob.add(order_a2)

        # 5. Verify Best Ask Price
        self.assertEqual(self.lob.asks.best_price(), 10.10, "Best ask price should be 10.10")

    def test_spread_and_mid_price(self):
        """Tests the computed properties for spread and mid-price."""
        
        # Should be None initially
        self.assertIsNone(self.lob.spread, "Spread should be None with empty book.")
        self.assertIsNone(self.lob.mid_price, "Mid price should be None with empty book.")
        
        # Setup book with a best bid and best ask
        self.lob.add(Order(price=50.00, volume=10, is_bid=True))
        self.lob.add(Order(price=50.01, volume=10, is_bid=False))
        
        # Spread = Best Ask (50.01) - Best Bid (50.00) = 0.01
        self.assertEqual(round(self.lob.spread, 2), 0.01, "Spread calculation is incorrect.")
        
        # Mid-Price = (50.01 + 50.00) / 2 = 50.005
        self.assertAlmostEqual(self.lob.mid_price, 50.005, places=5, msg="Mid price calculation is incorrect.")
        
    def test_order_cancellation(self):
        """Tests that a specific order can be cancelled and removes the level if empty."""
        
        # Add a bid to be cancelled (ID 0)
        cancel_order = Order(price=10.0, volume=100, is_bid=True) 
        self.lob.add(cancel_order)
        # Add a placeholder order (ID 1)
        placeholder_order = Order(price=9.9, volume=50, is_bid=True) 
        self.lob.add(placeholder_order)
        
        # Verify the order exists and best price is 10.0
        self.assertIn(cancel_order.id, self.lob.order_map)
        self.assertEqual(self.lob.bids.best_price(), 10.0)

        # Perform the cancellation (This level should now be empty)
        self.lob.cancel(cancel_order.id)

        # 1. Verify order is removed from the global map
        self.assertNotIn(cancel_order.id, self.lob.order_map, "Cancelled order should be removed from order map.")
        
        # 2. Verify the best price automatically moves to 9.9 because 10.0 level is empty
        self.assertEqual(self.lob.bids.best_price(), 9.9, "Best price should automatically move to 9.9 after 10.0 level is emptied.")
        
        # 3. Verify cancelling a non-existent ID raises KeyError (Assuming fix is applied)
        with self.assertRaises(KeyError):
            self.lob.cancel(999) 

# This block allows you to run the tests directly in the notebook cell.
if __name__ == '__main__':
    print("Running LOB Tests...")
    # This line runs the tests and prints results in a notebook friendly way
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


Running LOB Tests...
